### This code requires the dictionaries extracted with the 'loading_in_tar' code. From these dictionaries, we extract the neural data for a given brain region. A filter is set to only take spikes that were given a phy annotation of 2, indicating a good sort. 

In [ ]:
## getting just VISp data (V1)
# end result: dictionary with keys: session and values: spike indices for the spikes that were in ROI

import numpy

vis_ctx = {}
sessions = np.arange(1, 41)

for s in sessions:
    print('session:', s)
    # find which channels are in V1
    rows = np.shape(channel_brainLocations[s])[0] # how many channels in this session
    channels = [] 
    for r in range(rows):
        if channel_brainLocations[s][r]=='VISp': # this string can be modified to find data for any region
            channels.append(r)
        else:
            continue
    if len(channels)<1:
        print('this session had no recordings in ROI')
        continue
    channels = np.asarray(channels)
    
    # now find all clusters who's peak value was at this channel
    clusters = []
    for chan in channels:
        more_rows = np.shape(clusters_peakChannel[s])[0] # how many clusters in this session
        for r2 in range(more_rows):
            if int(np.squeeze(clusters_peakChannel[s][r2])) == chan and int(np.squeeze(clusters_phy_annotation[s][r2])) == 2:
                clusters.append(r2)
            else:
                continue
        else:
            continue
    #print(clusters)
    clusters = np.asarray(clusters)
    
    # now find all spikes who were included in this cluster
    spikes = []
    for clus in clusters:
        print('cluster', clus, 'of', len(clusters))
        most_rows = np.shape(spikes_clusters[s])[0] # how many spikes in this session
        for r3 in range(most_rows):
            if int(np.squeeze(spikes_clusters[s][r3])) == clus:
                spikes.append(r3)
                
    spikes = np.asarray(spikes)
    print(spikes)
    vis_ctx[s]=spikes

### If you want to save the resulting dictionary to an .npy file, here's the code for that.

In [ ]:
# EXPORT V1 SPIKE DICTIONARY to file

os.chdir('/your/filepath/here')
import pickle

file_to_write = open("V1_spikes.npy", "wb")
pickle.dump(vis_ctx, file_to_write)

### Now, we need to make a new dictionary for the spike timestamps that only has the values for spikes in V1.

In [7]:
import os
import numpy as np

os.chdir('/Users/srbindas/Documents/OHSU/NMA/pickles')
vis_ctx = np.load("V1_spikes.npy", allow_pickle=True)
spikes_times = np.load('spikes_times.npy', allow_pickle=True)

# initialize dictionary with the same keys as vis_ctx
vis_timestamps = {}
keyList = vis_ctx.keys()
for i in keyList: 
    vis_timestamps[i] = None
    
for (keyTS, valueTS) in vis_timestamps.items():
    print('session', keyTS)
    spike_indices = vis_ctx[keyTS]
    spiketimes = []
    for key, value in vis_ctx.items():
        for v in value:  
            spiketime = spikes_times[key][v]
            spiketimes.append(spiketime)
        vis_timestamps[key] = np.asarray(spiketimes)

print('done!')

### Lastly, we will make a dictionary containing the cluster each V1 neuron belongs to. 

In [2]:
import os
import numpy as np

os.chdir('/Users/srbindas/Documents/OHSU/NMA/pickles')
vis_ctx = np.load("V1_spikes.npy", allow_pickle=True)
spikes_clusters = np.load('spikes_clusters.npy', allow_pickle=True)

# initialize dictionary with the same keys as vis_ctx
vis_clusters = {}
keyList = vis_ctx.keys()
for i in keyList: 
    vis_clusters[i] = None
    
for (keyC, valueC) in vis_clusters.items():
    print('session', keyC)
    spike_indices = vis_ctx[keyC]
    spike_clusters = []
    for key, value in vis_ctx.items():
        for v in value:  
            spikeclus = spikes_clusters[key][v]
            spike_clusters.append(spikeclus)
        vis_clusters[key] = np.asarray(spike_clusters)

print('done!')

session 5
session 9
session 15
session 16
session 18
session 21
session 25
session 27
session 30
session 34
session 35
session 36
done!


In [6]:
# EXPORT DICTIONARY to file

os.chdir('/Users/srbindas/Documents/OHSU/NMA/pickles')
import pickle

file_to_write = open("V1_clusters.npy", "wb")
pickle.dump(vis_clusters, file_to_write)